<a href="https://colab.research.google.com/github/Sihan-A/PyTorch/blob/main/PyTorch_Lightning/PL_Python_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Lighning Tutorial

Youtube: [PyTorch Lightning](https://www.youtube.com/watch?v=Hgg8Xy6IRig)

Github: [code](https://github.com/python-engineer/pytorch-examples/blob/master/pytorch-lightning/lightning.py)

In [1]:
# install pytorch lightning
!pip install pytorch-lightning

     |████████████████████████████████| 923 kB 10.1 MB/s 
     |████████████████████████████████| 282 kB 41.6 MB/s 
     |████████████████████████████████| 119 kB 44.6 MB/s 
     |████████████████████████████████| 636 kB 51.9 MB/s 
     |████████████████████████████████| 829 kB 59.2 MB/s 
     |████████████████████████████████| 1.3 MB 43.8 MB/s 
     |████████████████████████████████| 142 kB 74.7 MB/s 
     |████████████████████████████████| 294 kB 77.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=a3f2e17c043e923f84347574ca2361f72bc6108169518ab2fcaa91e3342fa36a
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-

In [2]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision.datasets import MNIST
from torchvision import transforms
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [3]:
# hyper-parameters
input_size = 784
hidde_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
class LitNeuralNet(pl.LightningModule):
    def __init__(self, input_size, hidde_size, num_classes):
        super().__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidde_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidde_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out
    
    def training_step(self, batch, batch_idx):
        images, labels = batch
        # flatten the input
        images = images.reshape(-1, 28*28)
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss,
                "log": tensorboard_logs}
    
    def train_dataloader(self):
        train_dataset = MNIST(root="./data", train=True,
                              transform=transforms.ToTensor(), download=True,)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                                  num_workers=2, shuffle=False,)
        return train_loader
    
    def val_dataloader(self):
        test_dataset = MNIST(root="./data", train=False,
                             transform=transforms.ToTensor(), download=True,)
        test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,
                                 num_workers=2, shuffle=False,)
        return test_loader
    
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        images = images.reshape(-1, 28*28)
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        return {"val_loss": loss}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_val_loss": avg_loss}
        return {"val_loss": avg_loss,
                "log": tensorboard_logs}
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),
                                lr=learning_rate)

In [5]:
model = LitNeuralNet(input_size, hidde_size, num_classes)
trainer = Trainer(max_epochs=num_epochs, fast_dev_run=False,gpus=1)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 392 K 
1 | relu | ReLU   | 0     
2 | l2   | Linear | 5.0 K 
--------------------------------
397 K     Trainable params
0         Non-trainable params
397 K     Total params
1.590     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
%load_ext tensorboard
!tensorboard --logdir logging_logs

2021-09-18 07:14:43.197127: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-18 07:14:43.511535: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-18 07:14:43.512170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL